In [1]:
import geopandas as gpd
import pandas as pd
from lakedata import georeader, waterquality
from shapely.geometry import Point
from numpy import isfinite
%config Completer.use_jedi = False

In [2]:
nhd_dir = "../datafiles/Shape/"
eco_dir = "../waeco/"

In [3]:
lakedata = georeader.getNHDWaterbodies(nhd_dir)
ecodata  = waterquality.loadWaterQualityData(eco_dir+'toxin2017_11_6.csv', eco_dir+'EIMLocation.csv')
ecodata = ecodata[isfinite(ecodata.Latitude_Decimal_Degrees) | isfinite(ecodata.Longitude_Decimal_Degrees)]

b'Skipping line 27: expected 10 fields, saw 11\nSkipping line 28: expected 10 fields, saw 11\nSkipping line 520: expected 10 fields, saw 11\nSkipping line 521: expected 10 fields, saw 11\nSkipping line 552: expected 10 fields, saw 11\nSkipping line 553: expected 10 fields, saw 11\nSkipping line 554: expected 10 fields, saw 11\nSkipping line 555: expected 10 fields, saw 11\nSkipping line 556: expected 10 fields, saw 11\nSkipping line 557: expected 10 fields, saw 11\nSkipping line 563: expected 10 fields, saw 11\nSkipping line 564: expected 10 fields, saw 11\nSkipping line 565: expected 10 fields, saw 11\nSkipping line 566: expected 10 fields, saw 11\nSkipping line 567: expected 10 fields, saw 11\nSkipping line 568: expected 10 fields, saw 11\nSkipping line 569: expected 10 fields, saw 11\nSkipping line 570: expected 10 fields, saw 11\n'


In [4]:
ecodata = ecodata.replace('<MDL', 0)

In [5]:
lakedata.head()

,PERMANENT_,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,AREASQKM,ELEVATION,REACHCODE,FTYPE,FCODE,...,DESCRIPTIO,HYDROGRAPH,INUNDATION,KCODE,OPERATIONA,PIPELINETY,POSITIONAL,RELATIONSH,RESERVOIRT,STAGE
0,43938735,2016-03-24,2,None,None,0.004416,None,17060108009772,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
1,51681191,2002-08-14,2,None,None,0.009000,None,17010306105790,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
2,51681387,2013-12-26,2,None,None,0.005190,None,17010306105796,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
3,52133257,2002-08-20,2,None,None,0.003000,None,17010214129309,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
4,57379485,2002-10-09,2,None,None,0.002000,None,17010305089159,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None


In [6]:
display(ecodata.head())
print(len(ecodata))

,County,Site,LabSampleNum,WRIANumber,CollectDate,Parameter,Toxin Concentration (microg/L),MDL (microg/L),ExceedInd,ScumInd,loc_idx,Latitude_Decimal_Degrees,Longitude_Decimal_Degrees
13,Benton,Yakima Delta,L68576-2,37.0,09/06/2017,Anatoxin-a,0,0.01,No,No,300,46.238860,-119.228810
14,Benton,Yakima Delta,L68576-2,37.0,09/06/2017,Microcystin,0.339,0.15,No,No,300,46.238860,-119.228810
15,Benton,Yakima Delta,L68429-1,37.0,08/14/2017,Anatoxin-a,0.010,0.01,No,Yes,300,46.238860,-119.228810
16,Benton,Yakima Delta,L68429-1,37.0,08/14/2017,Microcystin,0,0.15,No,Yes,300,46.238860,-119.228810
17,Chelan,Fish Lake,L63544-5,45.0,08/17/2015,Microcystin,0,0.16,No,Yes,103,47.835594,-120.703665


12124


In [7]:
eco_geom = [Point(xy) for xy in zip(ecodata.Longitude_Decimal_Degrees, ecodata.Latitude_Decimal_Degrees)]
ecodata = gpd.GeoDataFrame(data=ecodata, geometry=eco_geom, crs={'init': 'epsg:4326'})

In [8]:
ecodata['geometry'] = ecodata.geometry.buffer(0.01)
lakedata = lakedata.to_crs(ecodata.crs)
ecodata_withlake = gpd.sjoin(ecodata, lakedata, how='left')
# keep only rows with match
ecodata_withlake = ecodata_withlake[isfinite(ecodata_withlake.FCODE) == True]
ecodata_withlake.columns

Index(['County', 'Site', 'LabSampleNum', 'WRIANumber', 'CollectDate',
       'Parameter', 'Toxin Concentration (microg/L)', 'MDL (microg/L)',
       'ExceedInd', 'ScumInd', 'loc_idx', 'Latitude_Decimal_Degrees',
       'Longitude_Decimal_Degrees', 'geometry', 'index_right', 'PERMANENT_',
       'FDATE', 'RESOLUTION', 'GNIS_ID', 'GNIS_NAME', 'AREASQKM', 'ELEVATION',
       'REACHCODE', 'FTYPE', 'FCODE', 'SHAPE_LENG', 'SHAPE_AREA', 'VISIBILITY',
       'CANALDITCH', 'CONSTRUCTI', 'DESCRIPTIO', 'HYDROGRAPH', 'INUNDATION',
       'KCODE', 'OPERATIONA', 'PIPELINETY', 'POSITIONAL', 'RELATIONSH',
       'RESERVOIRT', 'STAGE'],
      dtype='object')

In [9]:
ecodata_withlake = ecodata_withlake.merge(lakedata.set_index(lakedata.PERMANENT_)[['geometry']], 
                                          left_on='PERMANENT_', 
                                          right_index=True, 
                                          how='left', 
                                          suffixes=('_pt', '_lake'))


In [10]:
ecodata_withlake.CollectDate = pd.to_datetime(ecodata_withlake.CollectDate)

In [11]:
recent_ecodata = ecodata_withlake[ecodata_withlake.CollectDate >= '2016-01-01']


In [12]:
pondslakes = recent_ecodata[recent_ecodata.FTYPE == 390].groupby(['Site', 'CollectDate'])
len(pondslakes)

1425

In [13]:
final = gpd.GeoDataFrame(crs=ecodata.crs)
# select for scum or not -- choosing toxin at random here. 

for row in pondslakes:
    #picking the first row that comes up. assuming scum presence is the same in a single site/date pair
    final = final.append(row[1].iloc[0])
    


In [14]:
final.geometry = final.geometry_lake
final.CollectDate = final.CollectDate.astype(str)
final = final.drop(['geometry_lake', 'geometry_pt'], axis=1)
final.sample(n=500).to_file("scum_500.shp")

In [15]:

final.columns

Index(['AREASQKM', 'CANALDITCH', 'CONSTRUCTI', 'CollectDate', 'County',
       'DESCRIPTIO', 'ELEVATION', 'ExceedInd', 'FCODE', 'FDATE', 'FTYPE',
       'GNIS_ID', 'GNIS_NAME', 'HYDROGRAPH', 'INUNDATION', 'KCODE',
       'LabSampleNum', 'Latitude_Decimal_Degrees', 'Longitude_Decimal_Degrees',
       'MDL (microg/L)', 'OPERATIONA', 'PERMANENT_', 'PIPELINETY',
       'POSITIONAL', 'Parameter', 'REACHCODE', 'RELATIONSH', 'RESERVOIRT',
       'RESOLUTION', 'SHAPE_AREA', 'SHAPE_LENG', 'STAGE', 'ScumInd', 'Site',
       'Toxin Concentration (microg/L)', 'VISIBILITY', 'WRIANumber',
       'index_right', 'loc_idx', 'geometry'],
      dtype='object')